# Scalar Example with Sigmoids
Verify the ability to learn the function
$$
f(x) = e^{-x^2/2}
$$
using Adaptive Random Fourier Features (ARFF) with sigmoid activation functions.

In [ ]:
using Plots
using Random
using Statistics
using Distributions
using Printf
using LinearAlgebra
using ARFF
using SpecialFunctions
using LaTeXStrings

In [ ]:
default(lw=2,markersize = 6,
    xtickfont=font(12), ytickfont=font(12), 
    guidefont=font(14), legendfont=font(12),titlefont=font(12))

In [ ]:
f(x) = exp(-0.5 * (x^2));

__NOTE__ In this implementation, data points, $(x,y)$, $x$ is stored as a $d$-dimensional array, even if $d=1$.  This allows the code to more easily work across different $d$.  Also, we have padded `x` with a `1`, so that we can have nonzero activation functions.

In [ ]:
n_x = 100; # number of sample points
d = 2;
Random.seed!(100);
x = [[4*rand(),1] for _ in 1:n_x]; # generate n_x sample points, storying them as an array of 1D points
y = [f(x_[1]) for x_ in x];


# store data in DataSet structure
data = DataSet(x,y);

scatter([x_[1] for x_ in x], y, label="Sample Points")
xx = LinRange(0, 4, 100);
plot!(xx, f.(xx), label=L"f(x)")
xlabel!(L"x")

For a given number of Fourier modes,$K$, initialize a random model

In [ ]:
@show K = 2^4;
Random.seed!(200)
# lorentz(z) = 1/(1+z^2);
hat(z) = exp(-abs(z));

F0 = FourierModel([1. *randn() for _ in 1:K],  
    [1.0 * randn(d) for _ in 1:K], 
    SigmoidActivation);

Set the training parameters, and store in an options data structure

In [ ]:
δ = 0.1; # rwm step size
λ = 1e-6; # regularization
n_epochs = 10^3; # total number of iterations
n_ω_steps = 10; # number of steps between full β updates
n_burn = n_epochs ÷ 10;
γ = optimal_γ(d);
ω_max =Inf;
adapt_covariance = true;

Σ0 = Float64[1 0; 0 1];

function reg_β_solver!(β, S, y, λ)
    N = length(y);
    β .= (S' * S + λ * N *I) \ (S' * y)

end

β_solver! = (β, S, y, ω)-> reg_β_solver!(β, S, y, λ);

opts = ARFFOptions(n_epochs, n_ω_steps, δ, n_burn, γ, ω_max,adapt_covariance, 
    β_solver!, ARFF.mse_loss);

Train the model

In [ ]:
Random.seed!(1000);
F = deepcopy(F0);
Σ_mean, acceptance_rate, loss= train_rwm!(F, data, Σ0, opts, show_progress=true);

In [ ]:
@show Σ_mean;

In [ ]:
plot(1:length(loss), loss, yscale=:log10, xscale=:log10, label="")
xlabel!("Epoch")
ylabel!("Loss")

In [ ]:
scatter(1:length(acceptance_rate), acceptance_rate, xscale=:log10)
xlabel!("Epoch")
ylabel!("Acceptance Rate")

In [ ]:
xx = LinRange(0, 4, 500);
scatter([x_[1] for x_ in x], y, label="Sample Points", legend=:right)
plot!(xx, f.(xx), label = "Truth" )

plot!(xx, [F([x_, 1]) for x_ in xx],label="Learned Model")

xlabel!(L"x")

In [ ]:
abs(F([0., 1])-1)<1e-2

In [ ]:
scatter(data.y,F.(data.x),label="Data")
xx = LinRange(0,1,100);
plot!(xx, xx, ls=:dash, label="")
xlabel!("Truth")
ylabel!("Prediction")